# Pythonopdracht week 6: CNN

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-Images-from-Data-Set" data-toc-modified-id="Loading-Images-from-Data-Set-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading Images from Data Set</a></span></li><li><span><a href="#Define-a-Convolutional-Neural-Network" data-toc-modified-id="Define-a-Convolutional-Neural-Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define a Convolutional Neural Network</a></span></li><li><span><a href="#Training-the-Network" data-toc-modified-id="Training-the-Network-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training the Network</a></span></li><li><span><a href="#Testing-the-Network" data-toc-modified-id="Testing-the-Network-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing the Network</a></span></li><li><span><a href="#Improving-the-Network" data-toc-modified-id="Improving-the-Network-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Improving the Network</a></span></li></ul></div>

**For this notebook, you will both answer questions and do some programming. Please write answes in green in the given Answer cells. Please write code where instructed.**

This notebook is an adaptation of a demo notebook in the Pytorch documentation. Please be patient while running the code in the notebook. I haven't used any GPU that might be available on your computer. Training two epochs on my computer takes more than a minute (yes not too fast...).

In order to run the code you need to install pytorch on your system. The modules `torch` and `torchvision` should be installed. See the [PyTorch](https://pytorch.org/) website how to install Pytorch on your machine.

The [PyTorch](https://pytorch.org/) website is invaluable for understanding the code in this notebook. And it is a great source for further exploration of modern Machine Learning.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = None; 
# device = torch.device('cpu') # uncomment this line to explicitly set the device
if device == None:
    if torch.cuda.is_available():
        print("CUDA used")
        device = torch.device('cuda')
    else:
        print("CPU used")
        device = torch.device('cpu')

## Loading Images from the Data Set

PyTorch has some well known data sets 'build in', among others the CIFAR data set. In the code below we download (if needed) and read in the data set. In the process of defining the train and test set PyTorch allows us to transform the images on the fly. In this case we only convert the image to a tensor (the basic PyTorch data structure comparable with the ndarray in numpy) and we normalize the image (by subtracting 0.5 from each rgb value in the image and dividing by 0.5, as each image has rgb values in the range from 0 to 1 we end up with values in the range from -1 to +1). Normalization of the data is important in any neural network.

PyTorch loads a dataset into separate train and test sets. First we define the trainset.

In [ ]:
# Uncomment the following lines if you get a SSL certificate error to disable the verification.
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context (edited) 

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True, 
                                        download=True, 
                                        transform=transform)
print(trainset)

The trainset has an attribute ``data`` that contains the entire datamatrix in a numpy array.

In [ ]:
trainset.data.shape

i.e. 50000 images of 32x32 pixels each with an R, G and B value.

Also the labels are encoded in a simple Python list

In [ ]:
len(trainset.targets)

The target values are indices from 0 to 9 and the corresponding class names are encoded in the following list:

In [ ]:
classnames = trainset.classes
print(classnames)

In pure Python/Numpy we can now display one of the images with the target class name as title for the plot:

In [ ]:
i = 66
plt.imshow(trainset.data[i])
plt.axis('off')
plt.title(classnames[trainset.targets[i]]);

Later on we will define a ``DataLoader`` that makes it easy to load a small number of (image, target) tuples (a batch) needed in one gradient descent step in the stochastic gradient learning process. While loading the data the data in the dataset is transformed on the fly. For this dataset two transforms are used in sequence. First the ``ToTensor`` transform that takes an HxWxC (with C=3) numpy array into a PyTorch tensor of shape CxHxW which is the standard for encoding color images in deep learning. The second transform in the sequence is a normalization (see the PyTorch documentation for ``torchvision.transforms.Normalize``).

<span style="color:red">**Question 1.1**</span>.

Given the normalization ``Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))`` and an original color value (r,g,b) what is the normalized color value (r',g',b')? What do all those values 0.5 represent?

<span style="color:Green">
    
**Answer**

</span>

In the same way we define and load the testset:

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False, 
                                       download=True, 
                                       transform=transform)

Then we define a DataLoader for both the train and testset. When defining a DataLoader we have to give the `batch_size` as argument for the class constructor. A DataLoader, being a Python iterable, makes it possible to retrieve batch_size arbitrary images from the data set.

In [ ]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=2)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=batch_size, 
                                         shuffle=False, 
                                         num_workers=2)

## Displaying Tensor Images

For displaying 'tensor images' we provide two helper functions:

In [ ]:
def imshow(img, ax=None):
    """imshow for Tensor Images obtained with a DataLoader
    from a data set. The images are of shape (3,M,N)
    and are in the range from -1 to +1"""
    img = (img - img.min()) / (img.max()-img.min())     # unnormalize
    npimg = img.numpy()
    if ax==None:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
    else:
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        
def imtable(imgs, titles, ncols, hsize=4, vsize=4):
    nrows = int(np.ceil(len(imgs)/ncols))
    fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*hsize, nrows*vsize))
    r, c = 0, 0
    for i, (im, label) in enumerate(zip(imgs, titles)):
        if nrows==1:
            ax = axs[c]
        elif ncols==1:
            ax = axs[r]
        else:
            ax = axs[r,c]
        imshow(im, ax=ax)
        ax.set_title(titles[i])
        ax.axis('off')
        c += 1
        if c == ncols:
            r += 1
            c = 0
    plt.tight_layout()
        
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)
imtable(images, [classnames[i] for i in labels], 4)


<span style="color:red">**Question 1.2**</span>.

If you would be random guessing the class of an image what would be the expected accuracy of the 'classifier'?

<span style="color:Green">
    
**Answer**
    
    
</span>

## Define a Convolutional Neural Network

We define the following CNN to classify the CIFAR images. Observe that first the functional processing blocks in the CNN are instantiated in the ``__init__`` function and then used in the ``forward`` method implementing the forward pass from input to output of the network.

The backpropagation pass to calculate the derivatives of the loss with respect to the data throughout the network is automagically done by PyTorch. Only when you as a programmer introduce new functional processing blocks you have to implement the backward pass yourself.

In [ ]:
class Net(nn.Module):
    def __init__(self, verbose=0):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.verbose = verbose
        
    def forward(self, x):
        # Note in the forward pass we have 'numbered'
        # the x's just to make it easier to collect
        # the code to print information about the network
        # further on in the code.
        x1a = self.conv1(x)
        x1 = self.pool1(F.relu(x1a))
        x2 = self.pool2(F.relu(self.conv2(x1)))
        x3 = torch.flatten(x2, 1) # flatten all dimensions except batch
        x4 = F.relu(self.fc1(x3))
        x5 = F.relu(self.fc2(x4))
        x6 = self.fc3(x5)
            
        if self.verbose:
            # Write the code to print the properties of the 
            # different stages in the CNN
            # Your code here
            self.verbose -= 1
        
        return x6

# with verbose=1 the 'debugging' code is printed only once 
# while running the forward pass multiple times.
net = Net(verbose=1).to(device=device)

Given the class definition for a network we can print an instantiation of that class to obtain an overview of the functional processing blocks that are defined. Note that the ``relu`` and ``flatten`` functions are not printed (but they are in the flow of processing blocks).

In [ ]:
print(net)

Note that the shapes of the data that is processed at all layers in the network is not printed. That is where we need 'your code' in the ``Net`` class.

The code below sets the verbose attribute of the ``Net`` instantiation to 1 and then feeds it a batch of images. The output is printed, which is not needed of course as it will be total nonsense as nothing is learned yet.

It should be noted that PyTorch automagically takes care of proper initialization of the parameters of each of the processing modules in the network. If you want to you can override the default initialization (read the docs for that).

In [ ]:
net.verbose=1 # when you run it you should set this to 1
images, labels = next(dataiter)
images = images.to(device)
labels = labels.to(device)
outputs = net(images)
print(outputs)

With the printed information you ought to be able to answer the following questions:

In a data graph the first part of this network looks like
<img src="https://rvdboomgaard.github.io/ComputerVision_LectureNotes/_static/cifar_cnn_incomplete.png"
     width=80%
     alt="data flow graph"
     style="float: center;" />

The CIFAR images are 32x32 color images, so $W_1 = H_1 = 32$ and $C_1 = 3$. 

Observe that in this common way to depict a CNN the blocks that are drawn corresponds with the data, the arrows correspond with the data processing modules. E.g. the first arrow corresponds with the sequence Conv1 --> ReLU --> Maxpool modules. 

The last data block in this CNN is a vector of 10 elements where each element equals the activation for one of the 10 classes. This last layer is obtained with a fully connected neural network from the layer before that which is also a vector but with 84 elements.

Note that if you want to know what the data shape is after just the first or second convolution you have to change the code in the ``forward`` method.

<span style="color:red">**Question 2.1**</span>

If you print the shape of tensor ``x`` in the ``forward`` method you will get ``torch.Size([4, 3, 32, 32])``. Why is the first dimension of size 4 in this tensor? Note that this size is NOT depicted in the above graph!

<span style="color:Green">
    
**Answer**
    
    
</span>

<span style="color:red">**Question 2.2**</span>

What is the shape of the tensor that results from ``conv1(x)``? What does it tell you about the convolution/correlation algorithm that is used? To find this experimentally you have to change the code in the ``forward`` method.

<span style="color:Green">
    
**Answer**
    
    
</span>

<span style="color:red">**Question 2.3**</span>

Complete the data graph. For each of data block (image blocks at the start and flattened arrays at the end) you should indicate the shape of the block. The blocks correspond with the tensors `x`, `x1` ... `x6` in the `forward` method. Either you should look into the documentation of the processing modules that are being used in the `__init__` function or you should run the code and print the shapes (when we are running/training the network).

<span style="color:Green">
    
**Answer**
    
    
</span>

<span style="color:red">**Question 2.4**</span>

In the `__init__` function the following processing modules are defined: `conv1`, `pool1`, `conv2`, `pool2`, `fc1`, `fc2` and `fc3`. Indicate where these modules are used in your data flow graph (i.e. you have to label the arrows in the graph with the (sequence of) operations associated with each arrow.

<span style="color:Green">
    
**Answer**
    
    
</span>

<span style="color:red">**Question 2.5**</span>

For each of processing modules `conv1`, `pool1`, `conv2`, `pool2`, `fc1`, `fc2` and `fc3` give the number of parameters that have to be learned (don't forget the bias parameters...)

<span style="color:Green">
    
**Answer**
    
    
</span>

**HINT**: The answer to question 2.5 can be computed by 'interrogating' the network. For instance for the `fc2` linear module we get:

The answer to the last question from the above cell can be computed by 'interrogating' the network. For instance for the `fc2` linear module we get:

In [ ]:
for p in net.fc2.parameters():
    print(p.shape)

So for this module there are two tensors defining the parameters. Note that an object of type `torch.Size` is not a tensor

<span style="color:red">**Question 2.6**</span>

What are these tensors? (please look back in your machine learning course notes).

<span style="color:Green">
    
**Answer**
    
    
</span>

<span style="color:red">**Question 2.7**</span>

How many parameters in total for this `fc2` module?

<span style="color:Green">
    
**Answer**
    
    
</span>

Below you have to write the code to calculate the total number of parameters to be learned in this network.

In [1]:
# Write the code to calculate the total number of parameters in the entire network
# Your code here

Later on in this notebook we revisit the interrogation of the network again to actually observe what parameter values are learned.

## Loss Function and Optimizer

To train the network we also need to define a loss function:

In [ ]:
loss_criterion = nn.CrossEntropyLoss()

When the network is fed input the output is obtained with ``outputs = net(inputs)``, then the outputs are compared with the target values (labels in our case) with the `CrossEntropyLoss` function. Read the PyTorch documentation to learn why this loss is well suited for a classification problem (and why the softmax function is not needed in this network).

Observe that although the output of the network is a 10 element vector (activation for each of the different classes) the target value is an integer id for the target class. The conversion to one-hot-encoding is implicitly done within the loss function (see the source code).

We also need an optimizer. A standard optimizer is PyTorch is the SGD (Stochastic Gradient Descent) optimizer.

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, dampening=0)

here `lr` is the learning rate. The momentum is used in the gradient descent procedure to 'remember' in which direction the gradient descent is running and not deviate too easily from that direction.

<span style="color:red">**Question 2.8**</span>

Read the explanation of the SGD optimizer in the PyTorch documentation.
1. Why the name **stochastic** gradient descent?
2. What value for the ``momemtum`` and `dampening` should be chosen to get the classical gradient descent?

<span style="color:Green">
    
**Answer**
    
    
</span>

## Training the Network

Training the network using a variant of the gradient descent algorithm boils down to:
1. feed a minibatch of examples to the network
2. run the forward pass, i.e. calculate the output through all its intermediate processing modules)
3. set all gradients (derivative of the loss with respect to all parameters) equal to zero
4. calculate the backward pass (to calculate all gradients
5. for each of the parameters make a step in the right direction and adapt the parameters accordingly.

This process is repeated until the minibatches have fed all examples in the training set to the learning algorithm. That completex one **epoch** in learning. Then this process is repeated until a specified number of epochs is done.

In the learning process we keep track of the loss on the learning set and the loss on the test set both of which are calculated after each epoch. To have some indication whether the network is learning during an epoch we also report the loss over the last batches fed to the netword for learning.

One epoch of learning is encaptured in the `train_loop` function

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, no_report=1000, device=None):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Move the data to GPU (if available)
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % no_report == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
        

While learning for several epochs we also calculate the performance on the test or validation set. This is encoded in the function `test_loop`.

In [ ]:
def test_loop(dataloader, model, loss_fn, device=None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The learning process then becomes:

In [ ]:
%%timeit -n1 -r1
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, net, loss_criterion, optimizer, device=device)
    test_loop(testloader, net, loss_criterion, device=device)
print("Done!")

In [ ]:
# Uncomment the following 2 lines to save your network to file
#PATH = './cifar_net_100e.pth'
#torch.save(net.state_dict(), PATH)
#
# when saved you can load it back into memory with
# (again uncomment the following 2 lines)
#net = Net().to(device)
#net.load_state_dict(torch.load(PATH))

We load some images from the test set, display these image with their ground truth label. 

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)
imtable(images, [classnames[i] for i in labels], 4)

Then we run these image through the network and print the outputs of the 10 output nodes for all images. Note that now we are not training the network Pytorch doesn't need to keep track of all the intermediate outputs of all modules in the network (in preparation of a backward pass) and we can tell Pytorch not to do that with `no_grad()`.

While learning we didn't use the `softmax` function (see the PyTorch documentation) that normalizes the output for the 10 classes such that the a posteriori class probability if modeled (that would not be what the loss function expects). Now we are using the network in the feed forward mode only we will apply the softmax function.

In [ ]:
with torch.no_grad():
    outputs = F.softmax(net(images.to(device)), dim=-1)
npoutputs = outputs.cpu().numpy()
print(npoutputs)

The predicted label will be the label with the largest output value.

In [ ]:
maxvalue, maxindex = torch.max(outputs, 1)
print('True     : ', ' '.join('%10s' % classnames[labels[j]]
                              for j in range(batch_size)))
print('Predicted: ', ' '.join('%10s' % classnames[maxindex[j]]
                              for j in range(batch_size)))

For such a small set of test images we run into the possibility that the classification is very bad. Let's test the network more carefully.

## Inspecting the Network

In general it is really hard to understand what a (convolutional) neural network is doing. What do the learned weights represent? The answer to that question is still largely unknown. There are several papers that describe what particular nodes in the network are responsive to. These methods are beyond the scope of a bachelor course.

An exception is the first layer in a CNN. The convolutions there directly operate on the (color) image data and like we are used to in computer vision the convolution kernels (weigh functions) can be visualized as images themselves.

The hypothesis in developing CNN's was that local structure in images carries the spatial information needed to interpret the images. So when visualizing the kernels from the first convolution layer we hope to see the network has learned to respond to all sorts of local structure patterns.

These patterns are then in layers downwards in the datastream combined to recognize larger structures in images.

Your task is to visualize the kernels in the first convolution module `conv1` as small images.

In [ ]:
# Here the code to visualize the kernels
# Your code here

## Improving the Network

Selecting a network architecture for a CNN is more of an art than a science. Simple experimenting with different number of channels, with different number of (convolution) layers can sometimes improve your network.

<span style="color:red">**Question 5.1**</span>

1. The obvious way to start improving your network is by training for more epochs. Only 2 epochs is really not much (but you need a fast computer or a GPU or a lot of patience). But you should be able to increase the number of epochs from 2 to 10 when you run the learning at night.

1. Change the number of channels in the first data (image) block after the input image (a block with 3 channels) from 10 to 20 and train and run the network again. Does it improve the performance? Again start with 2 epochs of learning.

1. Change the size of the convolution kernels in the `conv1` module from 5 to 3. Again start with 2 epochs of learning.

<span style="color:Green">
    
**Answer**
    
    
</span>

## Bonus Exercises

- The speed of learning can be dramaticall improved by moving the entire dataset to the gpu (in case you have enough RAM on the gpu). I had a hard time to achieve that. If you can find an elegant way without the need to rewrite the data loaders i would be very interested.

- When you can learn faster or have a faster machine or have more time you could learn for a lot of epochs (say 100) and then plot the loss and accuracy for both the train and test set as a function of the number of epochs. That may provide you with evidence that the network is overfitting and thus that some regularization techniques might be neededn. This is suggested by the webpages describing learning for the same dataset but then implemented in Keras/Tensorflow (see https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/). There the loss and accuracy as function of the number of epochs is shown as (in yellow the testset scores and in blue the trainset scores):
<img src="https://rvdboomgaard.github.io/ComputerVision_LectureNotes/_static/cifar_overfitting.png"
     width=50%
     alt="data flow graph"
     style="float: center;" />
The network used was very much like the one that has been used in this lab.

- In the above mentioned paper using Keras/Tensorflow the problem over overfitting was succesfully mitigated using DropOut's in the network. It would be nice to see if you can do the same for our network.